In [46]:
# imports
from pathlib import Path
import math

import astropy
import numpy as np
import pandas as pd
from astropy.coordinates import SkyCoord
from rich import print
from tqdm import tqdm

In [72]:
# functions

# define a function to check that the name given is usable by astropy
def check_astropy(name, ra, dec, safety: int = 5):
    try:
        position = SkyCoord.from_name(name)
        return (abs(ra - position.ra.degree) < safety and abs(dec - position.dec.degree) < safety)
    except Exception: return False

In [71]:
# find and load the .csv
csv_file = Path("NewTargetsGCVS.csv")
df = pd.read_csv(csv_file)

In [73]:
# create astropy column
df['is_astropy_name'] = df.apply(lambda row: check_astropy(row["name"], row["ra"], row["dec"]), axis=1)

In [91]:
# limiting variables
mag_lim = 14
period_lim_hours = 2
ra_low_lim = 11
ra_high_lim = 19
mag_range_lim = 0.51

# apply astropy selection conditions (astropy can find the object)
astropy_df = df[(df["is_astropy_name"] == True)]

# apply ra selection conditions (ra within 12 to 18 hours)
ra_df = astropy_df[(ra_low_lim < astropy_df["ra_hours"]) & (astropy_df["ra_hours"] < ra_high_lim)]

# apply period selection conditions (period > 30 mins)
high_period_df = ra_df[(ra_df["period_days"]*24 > period_lim_hours)]

# apply magnitude selection conditions (magnitude < mag_lim)
temp_df = high_period_df[(high_period_df["max_mag"] < mag_lim)]
magnitude_df = temp_df[(temp_df["min_mag"] < mag_lim)]

# dataframe for low mag_range stars
low_range_df = magnitude_df[(magnitude_df["mag_range"] <= mag_range_lim)]

In [97]:
# create several data frames for "binning" the mag ranges
low_mag_range_df = low_range_df[(low_range_df["mag_range"] <= 0.21)]
mid_mag_range_df = low_range_df[(0.21 < low_range_df["mag_range"]) & (low_range_df["mag_range"] <= 0.4)]
high_mag_range_df = low_range_df[(0.4 < low_range_df["mag_range"])]

In [102]:
low_mag_range_df.to_csv(Path("LowMagRangeStars.csv"))
mid_mag_range_df.to_csv(Path("MidMagRangeStars.csv"))
high_mag_range_df.to_csv(Path("HighMagRangeStars.csv"))